In [1]:
import pandas as pd
import numpy as np
import pygad as pg
import deap as dp
import random
from deap import base, creator, tools, gp, algorithms
import time
import matplotlib.pyplot as plt

In [2]:
oferta = pd.read_csv('oferta.csv')
asignaturas = pd.read_csv('asignaturas.csv')
formato_cargas = pd.read_csv('formato_cargas.csv')
plan_2016 = pd.read_csv('plan_2016.csv')
seriaciones = pd.read_csv('../seriacion.csv')

# Definición de funciones útiles

In [3]:
matriculas = formato_cargas.query('n_matricula > 160000000')['n_matricula'].unique()

## Función para obtener un Kardex aleatorio

In [4]:
def obtenerKardex(mat = -1):
    if(mat == -1):
        mat = np.random.choice(matriculas)
    return formato_cargas.query('n_matricula == ' + str(mat)).sort_values('periodo')

In [5]:
def obtenerCreditos(kardex):
    claves = kardex.query('promediofinal >= 7')['clave'].unique()
    totalCreditos = 0
    for clave in claves:
        if(clave[0:2] == 'LI' or clave[0:2] == 'TA' or clave[0:2] == 'AD'):
            continue
        totalCreditos += plan.query('clave == "' + clave + '"')['creditos'].values[0]
    return totalCreditos

In [6]:
def obtenerRecomendacionesUnicas(recomendaciones,umbral):
    recomendacionesFinal = []
    for rec in recomendaciones:
        recomendacionesFinal.append(list(set(rec)))
    recomendacionesFinal = list(np.unique(recomendacionesFinal))
    recomendacionesFinal.sort(key=lambda x: -obtenerDesempenoPonderado(x))

    return recomendacionesFinal

In [7]:
def verKardex():
    print('Matricula: ' + str(kardex.iloc[0]['n_matricula']))
    print('Carrera: ' + str(kardex.iloc[0]['carrera']))
    print('Creditos: ' + str(obtenerCreditos(kardex))) 
    print('Materias Reprobadas:')
    for clave in obtenerMateriasReprobadas(kardex):
        print('\t' + clave,plan.query('clave == "' + clave + '"')['nombre'].values[0])
    print('\n')
    periodos = kardex['periodo'].unique()
    for per in periodos:
        print('Periodo: ' + str(per))
        print('Clave\t\tPromedio\tNombre de asignatura')
        materias = kardex.query('periodo == ' + str(per))
        for i in range(len(materias)):
            if(materias.iloc[i]['promediofinal'] < 7):
                print('* ' + materias.iloc[i]['clave'] + '\t' + str(materias.iloc[i]['promediofinal']) + '\t\t' + materias.iloc[i]['asignatura'])
            else:
                print(materias.iloc[i]['clave'] + '\t\t' + str(materias.iloc[i]['promediofinal']) + '\t\t' + materias.iloc[i]['asignatura'])
        print('\n')
    

## Obtención de oferta útil

In [8]:
def materiaHaSidoAprobada(kardex,clave):
    if len(kardex.query('clave == "' + clave + '" and promediofinal >= 7')) == 0:
        return False
    else:
        return True

In [9]:
def respetaSeriacion(clave,kardex):
    if len(seriaciones.query('ser2 == "' + clave + '"')) == 0:
        return True
    else:
        #Se guarda en necesarias la lista de asignaturas necesarias para llevar la asignura "clave"
        necesarias = seriaciones.query('ser2 == "' + clave + '"')['ser1'].unique()
        
        #Se recorre la lista para comprobar si el estudiante ya pasó las asignaturas necesarias, en caso de que le falte 
        #al menos una, entonces no se respeta la seriación.
        for necesaria in necesarias:
            if not(materiaHaSidoAprobada(kardex,necesaria)):
                return False
        return True

In [10]:
#Filtra las materias que no cumplen las restricciones 1 y 2

def obtenerOfertaUtil(kardex,oferta):
    dias = ['Lunes','Martes','Miercoles','Jueves','Viernes']
    
    #Se eliminan las materias en la oferta que ya han sido aprobadas
    aprobadas = kardex.query('promediofinal >= 7')['clave'].unique()
    for clave in aprobadas:
        oferta = oferta.query('clave != "' + clave + '"')
        
    #Se eliminan las materias que el alumno no puede llevar por la seriación
    ofertaUtilIndex = set(oferta.index.values)
    for idx in oferta.index:
        if not(respetaSeriacion(oferta.loc[idx]['clave'],kardex)):
            ofertaUtilIndex.remove(idx)
            
    #Se eliminan prácticas profesionales y proyecto terminal
    for i in range(len(oferta['clave'].values)):
        clave = oferta['clave'].values[i]
        if clave[0:3] == 'PID' or clave == 'IT0427':
            try:
                ofertaUtilIndex.remove(oferta.index.values[i])
            except:
                continue
    ofertaUtil = oferta.loc[list(ofertaUtilIndex)]
    
    ofertaUtil = pd.merge(ofertaUtil,plan,how='left',on='clave')[['clave','ciclos','Nombre','Maestro','Lunes','Martes','Miercoles','Jueves','Viernes']]
    
    #Se eliminan materias de elección libre de primer y segundo ciclo
    if(materiaHaSidoAprobada(kardex,'IL0102')):
        ofertaUtil = ofertaUtil.query('clave != "ID0160"')
    if(materiaHaSidoAprobada(kardex,'ID0160')):
        ofertaUtil = ofertaUtil.query('clave != "IL0102"')
    
    if(materiaHaSidoAprobada(kardex,'IT0103')):
        ofertaUtil = ofertaUtil.query('clave != "ID0161"')
    if(materiaHaSidoAprobada(kardex,'ID0161')):
        ofertaUtil = ofertaUtil.query('clave != "IT0103"')
        
    if(materiaHaSidoAprobada(kardex,'ID0264')):
        ofertaUtil = ofertaUtil.query('clave != "ID0262"')
    if(materiaHaSidoAprobada(kardex,'ID0262')):
        ofertaUtil = ofertaUtil.query('clave != "ID0264"')
        
    if(materiaHaSidoAprobada(kardex,'ID0263')):
        ofertaUtil = ofertaUtil.query('clave != "ID0265"')
    if(materiaHaSidoAprobada(kardex,'ID0265')):
        ofertaUtil = ofertaUtil.query('clave != "ID0263"')
        
    #Si la disponibilidad de horario es una restricción, entonces elimina las materias que violen la restricción
    if(disponibilidadComoRestriccion):
        indices = set(ofertaUtil.index)
        indicesUtiles = indices.copy()
        for dia in dias:
            for i in indices:
                if ofertaUtil.loc[i][dia] == '-':
                    continue
                
                horaInicio = int(ofertaUtil.loc[i][dia][0:2])
                horaFin = int(ofertaUtil.loc[i][dia][6:8])
                
                for hora in range(horaInicio,horaFin):
                    if not(disponibilidad.query('hora == ' + str(hora))[dia].values[0]):
                        if i in indicesUtiles:
                            indicesUtiles.remove(i)
        indicesUtiles = list(indicesUtiles)
        ofertaUtil = ofertaUtil.loc[indicesUtiles]
    
    return ofertaUtil

## Generación de carga aleatoria válida

En esta sección creamos las funciones "generarCargaValida" la cual retorna una string de bits representando una carga académica con base a la oferta útil. También se crea la función "obtenerDatosCarga" que nos ayuda a obtener los datos de una carga académica con base a la oferta útil y a una string de bits que denominamos "bits de carga"

In [11]:
def generarSolucionAleatoria(ofertaUtil):
    solucion = [-1] * 9
    for i in range(9):
        gen = np.random.randint(-1,len(ofertaUtil))
        if gen == -1:
            solucion[i] = -1
        else:
            solucion[i] = ofertaUtil.index[gen]
    return solucion

In [12]:
def obtenerDatosCarga(solucion):
    solucionU = np.array(solucion)
    solucionU = np.unique(solucionU)
    solucionU = solucionU[solucionU >= 0]
    
    return ofertaUtil.loc[solucionU]

In [13]:
def comprobarTraslapacion(solucion,ofertaUtil):
    datosCarga = obtenerDatosCarga(solucion)
    dias = ['Lunes','Martes','Miercoles','Jueves','Viernes']
    
    for dia in dias:
        horarioDia = datosCarga.sort_values(dia)[dia].values
        for i in range(len(horarioDia)):
            if horarioDia[i] == '-':
                continue
            horaInicioI = int(horarioDia[i][0:2])
            horaFinI = int(horarioDia[i][6:8])
            
            for j in range(i+1,len(horarioDia)):
                horaInicioJ = int(horarioDia[j][0:2])
                horaFinJ = int(horarioDia[j][6:8])
                
                if not(horaFinJ  <= horaInicioI or horaInicioJ >= horaFinI):
                    return True
    return False

In [14]:
def esValido(solucion,ofertaUtil):
    #Si se repite una materia es inválido
    datosCarga = obtenerDatosCarga(solucion)
    if len(datosCarga['clave'].unique()) < len(datosCarga):
        return False
    #Si se traslapan dos materias es inválido
    if comprobarTraslapacion(solucion,ofertaUtil):
        return False
    #Si son menos de 3 materias es inválido
    solucionSet = set(solucion)
    if -1 in solucionSet:
        solucionSet.remove(-1)
    if(len(solucionSet)<3):
        return False
    
    #Si se llevan dos materias de elección libre del ciclo 1 y 2
    clavesSet = set(datosCarga['clave'])
    
    if 'IL0102' in clavesSet and 'ID0160' in clavesSet:
        return False
    if 'IT0103' in clavesSet and 'ID0161' in clavesSet:
        return False
    if 'ID0264' in clavesSet and 'ID0262' in clavesSet:
        return False
    if 'ID0263' in clavesSet and 'ID0265' in clavesSet:
        return False
    
    return True

In [15]:
def generarSolucionValida(ofertaUtil):
    bitsDeCarga = generarSolucionAleatoria(ofertaUtil)
    
    while not(esValido(bitsDeCarga,ofertaUtil)):
        bitsDeCarga = generarSolucionAleatoria(ofertaUtil)
    
    return bitsDeCarga

In [16]:
def obtenerHorario(carga):
    primeraHoraMinima = 24
    ultimaHoraMaxima = 0
    datosCarga = obtenerDatosCarga(carga)
    horario = pd.DataFrame({
        'Hora': ['7:00-8:00','8:00-9:00','9:00-10:00','10:00-11:00','11:00-12:00','12:00-13:00','13:00-14:00','14:00-15:00','15:00-16:00','16:00-17:00','17:00-18:00','18:00-19:00','19:00-20:00','20:00-21:00','21:00-22:00'],
        'Lunes': ['-','-','-','-','-','-','-','-','-','-','-','-','-','-','-'],
        'Martes': ['-','-','-','-','-','-','-','-','-','-','-','-','-','-','-'],
        'Miercoles': ['-','-','-','-','-','-','-','-','-','-','-','-','-','-','-'],
        'Jueves': ['-','-','-','-','-','-','-','-','-','-','-','-','-','-','-'],
        'Viernes': ['-','-','-','-','-','-','-','-','-','-','-','-','-','-','-'],
    })
    dias = ['Lunes','Martes','Miercoles','Jueves','Viernes']
    for dia in dias:
        for i in range(len(datosCarga)):
            if datosCarga[dia].iloc[i] == '-':
                continue

            horaInicio = int(datosCarga.iloc[i][dia][0:2])
            horaFin = int(datosCarga.iloc[i][dia][6:8])
            
            primeraHoraMinima = min(primeraHoraMinima,horaInicio)
            ultimaHoraMaxima = max(ultimaHoraMaxima,horaFin)
            
            nombre = datosCarga.iloc[i]['Nombre']

            for hora in range(horaInicio,horaFin):
                horario.loc[hora-7,dia]=nombre
    return horario[(primeraHoraMinima-7):(ultimaHoraMaxima-6)]

# Definición de funciones de utilidad y de costo

## Utilidad de carga académica con base en cantidad ideal de materias

In [17]:
def UpCM(solucion):
    if(cantidadMateriasIdeal == 0):
        return 1
    
    diferenciaMaxima = max(cantidadMateriasIdeal-3,9-cantidadMateriasIdeal)
    solucionSet = set(solucion)
    
    if -1 in solucionSet:
        solucionSet.remove(-1)
        
    tamanoCarga = len(solucionSet)
    separacion = abs(tamanoCarga-cantidadMateriasIdeal)
    
    return 1 - (separacion / diferenciaMaxima)

## Utilidad de carga académica con base en las materias reprobadas

In [18]:
def obtenerMateriasReprobadas(kardex):
    materiasReprobadas = kardex.query('promediofinal < 7')['clave'].unique()
    materiasReprobadasFinal = []
    for clave in materiasReprobadas:
        aprobado = len(kardex.query('clave == "' + clave + '" and promediofinal >= 7'))
        if aprobado == 0 and clave[0:2] != 'LI':
            materiasReprobadasFinal.append(clave)
    return materiasReprobadasFinal

In [19]:
def UpMR(kardex,solucion,ofertaUtil):
    materiasReprobadas = obtenerMateriasReprobadas(kardex)
    if len(materiasReprobadas) == 0:
        return 1
    datosCarga = obtenerDatosCarga(solucion)
    utilidadTotal = 0
    
    for clave in materiasReprobadas:
        cargado = len(datosCarga.query('clave == "' + clave + '"'))
        if cargado == 1:
            utilidadTotal += 1
            
    
    #Normalización
    utilidad = (utilidadTotal)/(len(materiasReprobadas))
    return utilidad

## Utilidad de carga académica con base en el cierre de ciclos

In [20]:
def UpCC(solucion,ofertaUtil):
    base = 3
    utilidad = [base**3,base**2,base,1]
    utilidadTotal = 0
    
    menorCiclo = min(ofertaUtil['ciclos']) - 1
    
    datosCarga = obtenerDatosCarga(solucion)
    
    claves = datosCarga['clave'].unique()
    for i in range(len(claves)):
        if claves[i][0:2] == 'AD'or claves[i][0:2] == 'TA' or claves[i][0:2] == 'LI' or claves[i][0:2] == 'PI':
            continue
            
        ciclo = int(plan.query('clave == "' + claves[i] + '"')['ciclos']) - 1
        utilidadTotal += utilidad[ciclo - menorCiclo]
    
    utilidadMaxima = 0
    cantidadMateriasCiclo1 = len(ofertaUtil.query('ciclos == ' + str(menorCiclo+1))['clave'].unique())
    cantidadMateriasCiclo2 = len(ofertaUtil.query('ciclos == ' + str(menorCiclo+2))['clave'].unique())
    cantidadMateriasCiclo3 = len(ofertaUtil.query('ciclos == ' + str(menorCiclo+3))['clave'].unique())
    cantidadMateriasCiclo4 = len(ofertaUtil.query('ciclos == ' + str(menorCiclo+4))['clave'].unique())
    cantidadMateriasMaxima = 9
    
    m1 = m2 = m3 = m4 = 0
    
    if (cantidadMateriasMaxima - cantidadMateriasCiclo1) >= 0:
        m1 = cantidadMateriasCiclo1
        cantidadMateriasMaxima -= cantidadMateriasCiclo1
    else:
        m1 = 9
        cantidadMateriasMaxima = 0
    if (cantidadMateriasMaxima - cantidadMateriasCiclo2) >= 0:
        m2 = cantidadMateriasCiclo2
        cantidadMateriasMaxima -= cantidadMateriasCiclo2
    elif cantidadMateriasMaxima > 0:
        m2 = cantidadMateriasMaxima
        cantidadMateriasMaxima = 0
    if (cantidadMateriasMaxima - cantidadMateriasCiclo3) >= 0:
        m3 = cantidadMateriasCiclo3
        cantidadMateriasMaxima -= cantidadMateriasCiclo3
    elif cantidadMateriasMaxima > 0:
        m3 = cantidadMateriasMaxima
        cantidadMateriasMaxima = 0
    if (cantidadMateriasMaxima - cantidadMateriasCiclo4) >= 0:
        m4 = cantidadMateriasCiclo4
        cantidadMateriasMaxima -= cantidadMateriasCiclo4
    elif cantidadMateriasMaxima > 0:
        m4 = cantidadMateriasMaxima
        cantidadMateriasMaxima = 0
        
    
    utilidadMaxima = m1*utilidad[0] + m2*utilidad[1] + m3*utilidad[2] + m4*utilidad[3]
 
    #Normalización
    utilidadNorm = (utilidadTotal)/(utilidadMaxima)
    return utilidadNorm

## Costo por amplitud de horario

In [21]:
def CpAH(solucion):
    amplitudAceptable = 7
    
    datosCarga = obtenerDatosCarga(solucion)
    dias = ['Lunes','Martes','Miercoles','Jueves','Viernes']
    horaMin = 21
    horaMax = 7
    
    for dia in dias:
        for hora in datosCarga[dia]:
            if hora == '-':
                continue
            horaMin = min(int(hora[0:2]),horaMin)
            horaMax = max(int(hora[6:8]),horaMax)
            
    amplitud = (horaMax - horaMin)
    if amplitud <= amplitudAceptable:
        return 0
    amplitud -= amplitudAceptable
    
    return amplitud / (15 - amplitudAceptable)

## Costo de carga académica con base en las horas libres

#### NOTA: Tomar en cuenta las horas libres por día

In [22]:
def CpHL(solucion,ofertaUtil):
    dias = ['Lunes','Martes','Miercoles','Jueves','Viernes']
    costoTotal = 0
    hlMax = 0

    datosCarga = obtenerDatosCarga(solucion)

    for dia in dias:
        datosCarga = datosCarga.sort_values(dia)
        if datosCarga[dia].iloc[len(datosCarga)-2] == '-':
            continue

        ultimaHoraFin = 0
        for i in range(len(datosCarga)):
            if datosCarga.iloc[i][dia] == '-':
                continue
            if ultimaHoraFin == 0:
                ultimaHoraFin = int(datosCarga.iloc[i][dia][6:8])
                primeraHoraDia = int(datosCarga.iloc[i][dia][0:2])
                continue
            horaInicio = int(datosCarga.iloc[i][dia][0:2])
            costoTotal += (horaInicio - ultimaHoraFin)
            ultimaHoraFin = int(datosCarga.iloc[i][dia][6:8])
        hlMax += (ultimaHoraFin - primeraHoraDia - 2)

    if hlMax == 0:
        return 0
    #Normalización
    costo = (costoTotal)/(hlMax)
    return costo

## Costo de carga académica con base en la disponibilidad de horario del estudiante

Creamos 4 casos diferentes donde los estudiantes trabajan. Guardamos el horario de ocupación de los estudiantes en un dataframe por cada estudiante.

In [23]:
# El estudiante trabaja por las mañanas tiempo completo
disp_est_1 = pd.DataFrame({
    "hora": [7,8,9,10,11,12,13,14,15,16,17,18,19,20,21],
    "Lunes": [False,False,False,False,False,False,False,False,False,True,True,True,True,True,True],
    "Martes": [False,False,False,False,False,False,False,False,False,True,True,True,True,True,True],
    "Miercoles": [False,False,False,False,False,False,False,False,False,True,True,True,True,True,True],
    "Jueves": [False,False,False,False,False,False,False,False,False,True,True,True,True,True,True],
    "Viernes": [False,False,False,False,False,False,False,False,False,True,True,True,True,True,True],
})


# El estudiante trabaja por las tardes tiempo completo
disp_est_2 = pd.DataFrame({
    "hora": [7,8,9,10,11,12,13,14,15,16,17,18,19,20,21],
    "Lunes": [True,True,True,True,True,True,False,False,False,False,False,False,False,False,False],
    "Martes": [True,True,True,True,True,True,False,False,False,False,False,False,False,False,False],
    "Miercoles": [True,True,True,True,True,True,False,False,False,False,False,False,False,False,False],
    "Jueves": [True,True,True,True,True,True,False,False,False,False,False,False,False,False,False],
    "Viernes": [True,True,True,True,True,True,False,False,False,False,False,False,False,False,False],
})

# El estudiante trabaja por las mañanas medio tiempo
disp_est_3 = pd.DataFrame({
    "hora": [7,8,9,10,11,12,13,14,15,16,17,18,19,20,21],
    "Lunes": [True,False,False,False,False,False,True,True,True,True,True,True,True,False,False],
    "Martes": [True,False,False,False,False,False,True,True,True,True,True,True,True,False,False],
    "Miercoles": [True,False,False,False,False,False,True,True,True,True,True,True,True,False,False],
    "Jueves": [True,False,False,False,False,False,True,True,True,True,True,True,True,False,False],
    "Viernes": [True,False,False,False,False,False,True,True,True,True,True,True,True,False,False],
})
# El estudiante trabaja por las tardes medio tiempo
disp_est_4 = pd.DataFrame({
    "hora": [7,8,9,10,11,12,13,14,15,16,17,18,19,20,21],
    "Lunes": [True,True,True,True,True,False,False,False,False,False,True,True,True,True,True],
    "Martes": [True,True,True,True,True,False,False,False,False,False,True,True,True,True,True],
    "Miercoles": [True,True,True,True,True,False,False,False,False,False,True,True,True,True,True],
    "Jueves": [True,True,True,True,True,False,False,False,False,False,True,True,True,True,True],
    "Viernes": [True,True,True,True,True,False,False,False,False,False,True,True,True,True,True],
})
disp_est_5 = pd.DataFrame({
    "hora": [7,8,9,10,11,12,13,14,15,16,17,18,19,20,21],
    "Lunes": [True,True,True,True,True,True,True,True,True,True,True,True,True,True,True],
    "Martes": [True,True,True,True,True,True,True,True,True,True,True,True,True,True,True],
    "Miercoles": [True,True,True,True,True,True,True,True,True,True,True,True,True,True,True],
    "Jueves": [True,True,True,True,True,True,True,True,True,True,True,True,True,True,True],
    "Viernes": [True,True,True,True,True,True,True,True,True,True,True,True,True,True,True],
})

In [24]:
disponibilidadArr = [disp_est_1,disp_est_2,disp_est_3,disp_est_4,disp_est_5]

In [25]:
def CpDH(solucion,ofertaUtil,disponibilidad):
    disponibilidadTotal = sum(disponibilidad['Lunes']) + sum(disponibilidad['Martes']) + sum(disponibilidad['Miercoles']) + sum(disponibilidad['Jueves']) + sum(disponibilidad['Viernes'])
    if disponibilidadTotal == 75:
        return 0
    
    dias = ['Lunes','Martes','Miercoles','Jueves','Viernes']
    datosCarga = obtenerDatosCarga(solucion)
    costoTotal = 0

    for dia in dias:
        for i in range(len(datosCarga)):
            if datosCarga[dia].iloc[i] == '-':
                continue

            horaInicio = int(datosCarga.iloc[i][dia][0:2])
            horaFin = int(datosCarga.iloc[i][dia][6:8])

            for hora in range(horaInicio,horaFin):
                if not(disponibilidad.query('hora == ' + str(hora))[dia].values[0]):
                    costoTotal += 1
                    
    #Normalización
    costo = (costoTotal)/(75 - disponibilidadTotal)
    return costo

# Definición de la función de desempeño

In [26]:
def obtenerDesempenoPonderado(solucion):
    if not(esValido(solucion,ofertaUtil)):
        return 0
    
    upcc = UpCC(solucion,ofertaUtil)
    upmr = UpMR(kardex,solucion,ofertaUtil)
    upcm = UpCM(solucion)
    cpdh = CpDH(solucion,ofertaUtil,disponibilidad)
    cpah = CpAH(solucion)
    
    utilidades = {
        "upcc": (upcc * pesos["upcc"]),
        "upmr": (upmr * pesos["upmr"]),
        "upcm": (upcm * pesos["upcm"]),
        "cpdh": pesos["cpdh"] - (cpdh * pesos["cpdh"]),
        "cpah": pesos["cpah"] - (cpah * pesos["cpah"]),
    }
    
    return sum(utilidades.values())

In [27]:
def obtenerDesempeno(solucion):
    if not(esValido(solucion,ofertaUtil)):
        return 0,0,0,1,1
    
    upcc = UpCC(solucion,ofertaUtil)
    upmr = UpMR(kardex,solucion,ofertaUtil)
    upcm = UpCM(solucion)
    cpdh = CpDH(solucion,ofertaUtil,disponibilidad)
    cpah = CpAH(solucion)
    
    return upcc,upmr,upcm,cpdh,cpah,

In [28]:
def evaluarSolucion(solucion):
    desempeno = obtenerDesempeno(solucion)
    if(desempeno == 0):
        print('Solución no válida')
        return
    
    upcc = UpCC(solucion,ofertaUtil)
    upmr = UpMR(kardex,solucion,ofertaUtil)
    upcm = UpCM(solucion)
    cpdh = CpDH(solucion,ofertaUtil,disponibilidad)
    cpah = CpAH(solucion)
    
    print("La carga evaluada tiene:")
    print("\t" + str(round(100*upcc,2)) + "% de utilidad por las materias de primeros ciclos llevadas.")
    print("\t" + str(round(100*upmr,2)) + "% del total de las materias reprobadas del estudiante.") 
    print("\t" + str(round(100*upcm,2)) + "% de utilidad por cantidad ideal de materias del estudiante.") 
    print("\t" + str(round(100*cpdh,2)) + "% de costo por sobreposición con las horas donde el estudiante no está disponible.")
    print("\t" + str(round(100*cpah,2)) + "% de costo por el total de amplitud de horario.")

In [29]:
def on_generation(ga_instance):
    print("Generación: " + str(ga_instance.generations_completed))

In [30]:
def obtenerClase():
    gen = np.random.randint(-1,len(ofertaUtil))
    clase = -1 if gen == -1 else ofertaUtil.index[gen]
    return clase

In [31]:
def mutacionUniforme(solucion,prob):
    solucion = solucion.copy()
    for i in range(len(solucion)):
        if random.random() < prob:
            solucion[i] = toolbox.attr_int()
    return dp.creator.Individual(solucion),

In [32]:
def factorial(p):
    res = 1
    
    while p > 0:
        res *= p
        p -= 1
    
    return res

# DEAP

In [33]:
pesos = {
    "upcc": 2,    #Utilidad por cerrar ciclos
    "upmr": 1,    #Utilidad por materias reprobadas
    "upcm": 1,    #Utilidad por cantidad ideal de materias
    "cpdh": 2.5,  #Costo por disponibilidad de horario
    "cpah": 1.5,  #Costo por amplitud de horario
    "cphl": 1     #Costo por horas libres (no se está utilizando actualmente)
}

In [34]:
creator.create("FitnessMin", base.Fitness, weights=(1,1,1,-1,-1))
creator.create("Individual", list, fitness=creator.FitnessMin)

In [35]:
toolbox = base.Toolbox()
toolbox.register("attr_int", obtenerClase)
toolbox.register("individual", tools.initRepeat, creator.Individual,
                 toolbox.attr_int, n=9)
toolbox.register("population", tools.initRepeat, list, toolbox.individual)

toolbox.register("evaluate", obtenerDesempeno)
toolbox.register("mate", tools.cxUniform,indpb=0.5)
toolbox.register("mutate", mutacionUniforme,prob=0.15)

## Ejecución del algoritmo

In [36]:
pesos = {
    "upcc": 2,    #Utilidad por cerrar ciclos
    "upmr": 1,    #Utilidad por materias reprobadas
    "upcm": 1,    #Utilidad por cantidad ideal de materias
    "cpdh": 2.5,  #Costo por disponibilidad de horario
    "cpah": 1.5,  #Costo por amplitud de horario
    "cphl": 1     #Costo por horas libres (no se está utilizando actualmente)
}

disponibilidadComoRestriccion = False
disponibilidad = disponibilidadArr[4] #Disponibilidad completa de horario
plan = plan_2016

#kardex = obtenerKardex(170311242)
kardex = obtenerKardex(190311499).query('periodo < 202201')
ofertaUtil = obtenerOfertaUtil(kardex,oferta)
cantidadMateriasIdeal = 7

In [37]:
NOBJ = 5
K = 10
NDIM = NOBJ + K - 1
P = 12
H = factorial(NOBJ + P - 1) / (factorial(P) * factorial(NOBJ - 1))
BOUND_LOW, BOUND_UP = 0.0, 1.0
MU = int(H + (4 - H % 4))
NGEN = 25
CXPB = 1.0
MUTPB = 1.0
ref_points = tools.uniform_reference_points(NOBJ, P)

In [38]:
toolbox.register("select", tools.selNSGA3, ref_points=ref_points)

In [39]:
def main(seed=None):
    random.seed(seed)

    # Initialize statistics object
    stats = tools.Statistics(lambda ind: ind.fitness.values)
    stats.register("avg", np.mean, axis=0)
    stats.register("std", np.std, axis=0)
    stats.register("min", np.min, axis=0)
    stats.register("max", np.max, axis=0)

    logbook = tools.Logbook()
    logbook.header = "gen", "evals", "std", "min", "avg", "max"

    pop = toolbox.population(n=MU)

    # Evaluate the individuals with an invalid fitness
    invalid_ind = [ind for ind in pop if not ind.fitness.valid]
    fitnesses = toolbox.map(toolbox.evaluate, invalid_ind)
    for ind, fit in zip(invalid_ind, fitnesses):
        ind.fitness.values = fit

    # Compile statistics about the population
    record = stats.compile(pop)
    logbook.record(gen=0, evals=len(invalid_ind), **record)
    print(logbook.stream)

    # Begin the generational process
    for gen in range(1, NGEN):
        offspring = algorithms.varAnd(pop, toolbox, CXPB, MUTPB)

        # Evaluate the individuals with an invalid fitness
        invalid_ind = [ind for ind in offspring if not ind.fitness.valid]
        fitnesses = toolbox.map(toolbox.evaluate, invalid_ind)
        for ind, fit in zip(invalid_ind, fitnesses):
            ind.fitness.values = fit

        # Select the next generation population from parents and offspring
        pop = toolbox.select(pop + offspring, MU)

        # Compile statistics about the new population
        record = stats.compile(pop)
        logbook.record(gen=gen, evals=len(invalid_ind), **record)
        print(logbook.stream)

    return pop, logbook

In [40]:
pop,log = main()

gen	evals	std                                                     	min                            	avg                                                     	max                                                
0  	1824 	[0.11931864 0.19341729 0.16070886 0.19341729 0.04681465]	[ 0.    0.    0.   -0.    0.25]	[0.02360067 0.03892544 0.03166118 0.96107456 0.99273575]	[0.9047619 1.        1.        1.        1.       ]
1  	1824 	[0.16021323 0.25909696 0.21388216 0.25909696 0.07511787]	[ 0.  0.  0. -0. -0.]          	[0.04378133 0.07236842 0.05852522 0.92763158 0.98499178]	[0.9047619 1.        1.        1.        1.       ]
2  	1824 	[0.20363791 0.32695118 0.26970954 0.32695118 0.09901565]	[ 0.  0.  0. -0. -0.]          	[0.07403926 0.12171053 0.09813596 0.87828947 0.97423246]	[0.95238095 1.         1.         1.         1.        ]
3  	1824 	[0.24010134 0.38720853 0.32278202 0.38720853 0.12059595]	[ 0.  0.  0. -0. -0.]          	[0.11098058 0.18366228 0.14939693 0.81633772 0.96059485]	[0.9523

C:\Users\eddie\anaconda3\lib\site-packages\deap\tools\emo.py:610: RuntimeWarning: invalid value encountered in true_divide
  fn = (fitnesses - best_point) / (intercepts - best_point)


9  	1824 	[0.12093868 0.         0.1731028  0.         0.21113432]	[ 0.19047619  1.          0.         -0.         -0.        ]	[0.62450397 1.         0.81839364 0.         0.74876645]	[ 1.  1.  1. -0.  1.]                                   


C:\Users\eddie\anaconda3\lib\site-packages\deap\tools\emo.py:610: RuntimeWarning: invalid value encountered in true_divide
  fn = (fitnesses - best_point) / (intercepts - best_point)


10 	1824 	[0.12215775 0.         0.17243212 0.         0.2164094 ]	[ 0.19047619  1.          0.         -0.         -0.        ]	[0.64612051 1.         0.8238761  0.         0.69716283]	[ 1.  1.  1. -0.  1.]                                   


...

22 	1824 	[0.13811727 0.         0.17002134 0.         0.28450567]	[ 0.42857143  1.          0.25       -0.         -0.        ]	[0.75652673 1.         0.83689693 0.         0.42872807]	[ 1.  1.  1. -0.  1.]                                   


C:\Users\eddie\anaconda3\lib\site-packages\deap\tools\emo.py:610: RuntimeWarning: invalid value encountered in true_divide
  fn = (fitnesses - best_point) / (intercepts - best_point)


23 	1824 	[0.14509673 0.         0.16905217 0.         0.2926405 ]	[ 0.42857143  1.          0.25       -0.         -0.        ]	[0.74710213 1.         0.81647478 0.         0.39727248]	[ 1.  1.  1. -0.  1.]                                   


C:\Users\eddie\anaconda3\lib\site-packages\deap\tools\emo.py:610: RuntimeWarning: invalid value encountered in true_divide
  fn = (fitnesses - best_point) / (intercepts - best_point)


24 	1824 	[0.14862858 0.         0.16429449 0.         0.29140982]	[ 0.42857143  1.          0.25       -0.         -0.        ]	[0.7390873  1.         0.80372807 0.         0.36101974]	[ 1.  1.  1. -0.  1.]                                   


In [41]:
def plot_fitness(generation_solution_fitness):
    fig = plt.figure()
    plt.plot(generation_solution_fitness, linewidth=3)
    plt.title('Desempeño por generación')
    plt.xlabel('Generación')
    plt.ylabel('Desempeño')
    plt.show()

In [42]:
rec = obtenerRecomendacionesUnicas(pop,0)

C:\Users\eddie\anaconda3\lib\site-packages\numpy\lib\arraysetops.py:270: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  ar = np.asanyarray(ar)


In [43]:
len(rec)

1583

In [44]:
for i in range(10):
    print(obtenerDesempenoPonderado(rec[i]))

7.619047619047619
7.619047619047619
7.431547619047619
7.431547619047619
7.431547619047619
7.431547619047619
7.431547619047619
7.431547619047619
7.431547619047619
7.431547619047619


In [45]:
rec[0]

[3, 38, 14, 15, 16, 19, 30]

In [46]:
evaluarSolucion(rec[0])

La carga evaluada tiene:
	80.95% de utilidad por las materias de primeros ciclos llevadas.
	100% del total de las materias reprobadas del estudiante.
	100.0% de utilidad por cantidad ideal de materias del estudiante.
	0% de costo por sobreposición con las horas donde el estudiante no está disponible.
	0% de costo por el total de amplitud de horario.
